In [15]:
import os
from kafka import KafkaProducer
import pandas as pd
import json
import time
from dotenv import load_dotenv
import heapq

from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, window, lit
from pyspark.sql.types import StructType, StringType, TimestampType, IntegerType


load_dotenv(dotenv_path=os.path.expanduser("~/.bashrc"), override=True)
# os.environ["SHELL"] = "/usr/bin/bash"
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/opt/spark"


master_url = os.environ.get("SPARK_MASTER_URL")


trip_schema = StructType() \
    .add("trip_id", StringType()) \
    .add("timestamp", TimestampType()) \
    .add("event", StringType()) \
    .add("PULocationID", IntegerType()) \
    .add("DOLocationID", IntegerType()) \
    .add("passenger_count", IntegerType())

print("done")

done


In [18]:
print(master_url)
try:
    spark.stop()
except:
    pass
spark = SparkSession.builder \
    .appName("TaxiTripStream2") \
    .master(master_url) \
    .config("spark.ui.host", "0.0.0.0") \
    .config("spark.ui.port", "4040") \
    .config("spark.executor.memory", "2G") \
    .config("spark.driver.memory", "2G") \
    .config("spark.cores.max", "4") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1") \
    .getOrCreate()

 # .config("spark.driver.host", "10.186.0.4") \
 #.config("spark.driver.bindAddress","34.116.218.2")  \
#.config("spark.driver.port", "7078") \

spark://34.118.16.181:7077


In [19]:
df_raw = spark.readStream.format("kafka") \
    .option("kafka.bootstrap.servers", "34.118.105.77:9092") \
    .option("subscribe", "trips-start") \
    .option("startingOffsets", "earliest") \
    .load()

df_parsed = df_raw.selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), trip_schema).alias("data")) \
    .select("data.*")

query = df_parsed.writeStream \
    .option("truncate", False) \
    .outputMode("update") \
    .format("console") \
    .foreachBatch(lambda df, b: df.show()) \
    .start()

query.awaitTermination(20)
query.stop()

AnalysisException: Failed to find data source: kafka. Please deploy the application as per the deployment section of Structured Streaming + Kafka Integration Guide.

In [11]:
query.awaitTermination(10)
query.stop()

25/06/06 20:51:46 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/06/06 20:52:01 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/06/06 20:52:16 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


In [17]:


data = [("Alice", 25), ("Bob", 30), ("Charlie", 35)]
df = spark.createDataFrame(data, ["name", "age"])

# Run a simple transformation and action
df_filtered = df.filter(df.age > 28)
df_filtered.show()


25/06/06 22:56:37 WARN TaskSetManager: Lost task 0.0 in stage 0.0 (TID 0) (10.186.0.2 executor 3): java.io.IOException: org.apache.spark.SparkException: [INTERNAL_ERROR_BROADCAST] Failed to get broadcast_0_piece0 of broadcast_0
	at org.apache.spark.util.SparkErrorUtils.tryOrIOException(SparkErrorUtils.scala:42)
	at org.apache.spark.util.SparkErrorUtils.tryOrIOException$(SparkErrorUtils.scala:33)
	at org.apache.spark.util.Utils$.tryOrIOException(Utils.scala:94)
	at org.apache.spark.broadcast.TorrentBroadcast.readBroadcastBlock(TorrentBroadcast.scala:252)
	at org.apache.spark.broadcast.TorrentBroadcast.getValue(TorrentBroadcast.scala:109)
	at org.apache.spark.broadcast.Broadcast.value(Broadcast.scala:70)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:87)
	at org.apache.spark.TaskContext.runTaskWithListeners(TaskContext.scala:166)
	at org.apache.spark.scheduler.Task.run(Task.scala:141)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$4(Executor.scala:621)

+-------+---+
|   name|age|
+-------+---+
|    Bob| 30|
|Charlie| 35|
+-------+---+



In [16]:
df = spark.readStream.format("kafka") \
    .option("kafka.bootstrap.servers", "34.118.105.77:9092") \
    .option("subscribe", "trips-start") \
    .option("startingOffsets", "earliest") \
    .load()

df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)").writeStream \
    .format("console") \
    .option("truncate", False) \
    .outputMode("append") \
    .start() \
    .awaitTermination(10)

25/06/06 21:34:42 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/06/06 21:34:57 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/06/06 21:35:12 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/06/06 21:35:27 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/06/06 21:35:42 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/06/06 21:35:57 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure th

KeyboardInterrupt: 

25/06/06 21:39:57 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
